In [ ]:
# In this notebook we will explore the knapsack problem and its solution using dynamic programming and genetic algorithms

In [ ]:
# first we will implement the dynamic programming solution

In [ ]:
import random

In [ ]:
def knapsack(values, weights, capacity, verbose=False):
    num_items = len(values)

    dp = []
    for i in range(num_items + 1):
        dp.append([0] * (capacity + 1))

    for i in range(1, num_items + 1):
        for w in range(1, capacity + 1):
            if weights[i - 1] <= w:
                value_including_item = values[i - 1] + dp[i - 1][w - weights[i - 1]]
                value_excluding_item = dp[i - 1][w]
                dp[i][w] = max(value_including_item, value_excluding_item)
            else:
                dp[i][w] = dp[i - 1][w]

        if verbose and i % 100 == 0:
            print(f"Finished {i} items")

    return dp[num_items][capacity]

In [ ]:
# tests for the knapsack function

values = [60, 100, 110, 40, 50, 60]  
weights = [10, 20, 30, 5, 30, 5]   

test_capacities = [10, 20, 30, 50, 60, 70, 80, 90, 100]
for capacity in test_capacities:
    print(f"Capacity: {capacity} -> {knapsack(values, weights, capacity)}")

We can see a cool abstract depiction of knapsacks as a part of a genetic DNA sequence below

<img src="img/knapsack_pic.png" alt="knapsack abstract" width="400"/>

In [40]:
from time import time

def generate_knapsack(num_items):
    knapsack = [random.choice([0, 1]) for _ in range(num_items)]
    knapsack_capacity = 0
    for i in range(len(knapsack)):
        if knapsack[i] == 1:
            knapsack_capacity += weights[i]
    return knapsack

def calculate_fitness(knapsack, values, weights, capacity):
    total_value = 0
    total_weight = 0
    for i in range(len(knapsack)):
        if knapsack[i] == 1:
            total_value += values[i]
            total_weight += weights[i]
    if total_weight > capacity:
        return 0  # Penalize for exceeding capacity
    return total_value

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(knapsack, mutation_rate=0.05):
    for i in range(len(knapsack)):
        if random.random() < mutation_rate:
            knapsack[i] = 1 - knapsack[i]  # Flip the bit
    return knapsack

def knapsack_gen(values, weights, capacity, num_generations = 50, verbose=False, log_times = False):
    # Parameters
    num_items = len(values) 
    population_size = 1000
    times = {}

    population = [generate_knapsack(num_items) for _ in range(population_size)]

    for gen in range(num_generations):
        # Calculate fitness for each knapsack
        fitness_scores = [calculate_fitness(k, values, weights, capacity) for k in population]

        # Sort the population based on fitness and select the top knapsacks
        sorted_population = [x for _, x in sorted(zip(fitness_scores, population), reverse=True)]
        parents = sorted_population[:50]

        # Generate new population through crossover and mutation
        new_population = parents[:]
        while len(new_population) < population_size:
            parent1, parent2 = random.sample(parents, 2)
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([mutate(child1), mutate(child2)])

        population = new_population

        if verbose and gen % 10 == 0:
            best_solution = max(population, key=lambda k: calculate_fitness(k, values, weights, capacity))
            best_fitness = calculate_fitness(best_solution, values, weights, capacity)
            print(f"Generation: {gen} | Best fitness: {best_fitness}")
        
        if log_times and gen % 10 == 0:
            times[time()] = [gen, best_fitness]

    # Find the best solution at the end of the process
    best_solution = max(population, key=lambda k: calculate_fitness(k, values, weights, capacity))
    best_fitness = calculate_fitness(best_solution, values, weights, capacity)

    if log_times:
        return best_solution, best_fitness, times
    
    return best_solution, best_fitness

In [34]:
values = [60, 100, 110, 40, 50, 60]  
weights = [10, 20, 30, 5, 30, 5]   

test_capacities = [10, 20, 30, 50, 60, 70, 80, 90, 100]
for capacity in test_capacities:
    print(f"Capacity: {capacity} -> {knapsack_gen(values, weights, capacity)[1]}")

Capacity: 10 -> 100
Capacity: 20 -> 160
Capacity: 30 -> 200
Capacity: 50 -> 270
Capacity: 60 -> 310
Capacity: 70 -> 370
Capacity: 80 -> 370
Capacity: 90 -> 370
Capacity: 100 -> 420


We can see here we get the same solution as previously with our DP solution

Now let's start to test the limits of our DP solution

In [ ]:
random.seed(0)
values = [random.randint(10, 100) for _ in range(1000)]
weights = [random.randint(10, 100) for _ in range(1000)]
capacity = 25000

In [ ]:
start = time()
print(knapsack(values, weights, capacity))
print(f"Dynamic Programming Time: {time() - start}")

In [ ]:
start = time()
print(knapsack_gen(values, weights, capacity, 1000)[1])
print(f"Genetic Algorithm Time: {time() - start}")

Now lets really blow things up

In [35]:
random.seed(0)
values = [random.randint(10, 100) for _ in range(10000)]
weights = [random.randint(10, 100) for _ in range(10000)]
capacity = 275000

In [ ]:
start = time()
print(knapsack(values, weights, capacity, True))
print(f"Dynamic Programming Time: {time() - start}")

Optimal Solution is 420550
Took 690 seconds to run

In [41]:
start = time()
best_sol, best_fit, times = knapsack_gen(values, weights, capacity, 1000, True, True)
print(f"Genetic Algorithm Time: {time() - start}")